
# Ejemplo de Retrieval-Augmented Generation (RAG) con S-BERT y FAISS

En este notebook, vamos a implementar un ejemplo de Retrieval-Augmented Generation (RAG) utilizando **Sentence-BERT (S-BERT)** para la generación de embeddings y **FAISS** para la búsqueda eficiente. 

RAG combina técnicas de búsqueda con generación de respuestas para mejorar la calidad de las interacciones de los chatbots. Primero, recuperamos los documentos más relevantes de una base de datos y luego utilizamos esa información para generar respuestas más precisas.

El flujo de este ejemplo será el siguiente:
1. Crearemos un dataset con información almacenada en un array.
2. Generaremos embeddings usando S-BERT.
3. Almacenaremos estos embeddings usando la biblioteca **FAISS** para una búsqueda eficiente.
4. Crearemos un chatbot que solicitará al usuario una frase, generará su embedding y devolverá las respuestas más similares de la base de datos.


## Instalación e importación de librerías

In [ ]:

# Instalación de las bibliotecas necesarias
!pip install sentence-transformers faiss-cpu


In [ ]:
# Importar las librerías necesarias
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

## Generacíon del dataset de embeddings

In [19]:

# Crear un dataset simple con algunas frases de ejemplo
dataset = [
    "La inteligencia artificial está revolucionando el mundo.",
    "La tecnología de redes neuronales se usa en muchos campos.",
    "El procesamiento del lenguaje natural es una rama de la inteligencia artificial.",
    "Los sistemas de recomendación se basan en el aprendizaje automático.",
    "El Internet de las Cosas está cambiando la forma en que vivimos.",
    "La robótica es una combinación de ingeniería y tecnología avanzada.",
    "La visión por computadora se utiliza para analizar imágenes y videos."
]

# Inicializar el modelo de S-BERT para la generación de embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Generar embeddings para cada frase en el dataset
embeddings = model.encode(dataset)
print(f"Embeddings generados para el dataset: {embeddings.shape}")


Embeddings generados para el dataset: (7, 384)


In [4]:

# Crear un índice de FAISS para la búsqueda eficiente usando L2 (distancia euclidiana)
index = faiss.IndexFlatL2(embeddings.shape[1])

# Añadir los embeddings al índice
index.add(embeddings)
print(f"Total de vectores almacenados en el índice: {index.ntotal}")


Total de vectores almacenados en el índice: 7


## Creación y ejecución del chatbot

In [7]:

# Función para interactuar con el chatbot
def consultar_chatbot():
    # Solicitar al usuario una frase para buscar en el dataset
    consulta = input("Introduce una frase para buscar en el dataset: ")
    consulta = "¿Qué está revolucionando la IA?"

    # Generar el embedding de la frase del usuario
    consulta_embedding = model.encode([consulta])

    # Realizar la búsqueda en el índice FAISS para encontrar los elementos más similares
    D, I = index.search(consulta_embedding, k=3)  # Obtener los 3 elementos más similares

    # Mostrar los resultados al usuario
    print("\nResultados más similares encontrados en el dataset:")
    for i in range(len(I[0])):
        print(f"Frase: {dataset[I[0][i]]} (Similitud: {D[0][i]})")

# Ejecutar la función del chatbot
consultar_chatbot()


Resultados más similares encontrados en el dataset:
Frase: La inteligencia artificial está revolucionando el mundo. (Similitud: 26.722129821777344)
Frase: Los sistemas de recomendación se basan en el aprendizaje automático. (Similitud: 35.01416778564453)
Frase: El Internet de las Cosas está cambiando la forma en que vivimos. (Similitud: 40.01490020751953)


## Incorporación de un modelo generativo

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM


# Función para generar respuesta con GPT-2
def generar_respuesta_gen(pregunta):
    inputs = tokenizer(pregunta, return_tensors="pt", padding=True, truncation=True)
    # Generar respuesta utilizando el attention_mask
    outputs = model_gen.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=300,  # Permitir respuestas más largas
        num_return_sequences=1, 
        no_repeat_ngram_size=2, 
        temperature=0.7,  # Ajuste de la diversidad
        top_p=0.9,  # Nucleus sampling
        early_stopping=True
    )
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")
model_gen = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1")

In [30]:
# Función para interactuar con el chatbot
def consultar_chatbot():
    # Solicitar al usuario una frase para buscar en el dataset
    #consulta = input("Introduce una frase para buscar en el dataset: ")
    consulta = "¿En qué se basan los sistemas de recomendación?"
    
    # Generar el embedding de la frase del usuario
    consulta_embedding = model.encode([consulta])

    # Realizar la búsqueda en el índice FAISS para encontrar los elementos más similares
    D, I = index.search(consulta_embedding, k=3)  # Obtener los 3 elementos más similares

    # Mostrar los resultados al usuario
    print("\nResultados más similares encontrados en el dataset:")
    for i in range(len(I[0])):
        print(f"Frase: {dataset[I[0][i]]} (Diferencia: {D[0][i]})")

    # Usar BLOOMZ para generar una respuesta basada en la consulta
    contexto = dataset[I[0][0]]  # Utilizar la frase más relevante del dataset
    consulta_gpt = f"Contexto: {contexto}. Ahora responde a la pregunta: {consulta}"
    
    print("Consulta a BLOOMZ")
    print(consulta_gpt)
    
    respuesta_bloomz = generar_respuesta_gen(consulta_gpt)
    print("\nRespuesta generada por BLOOMZ:")
    print(respuesta_bloomz)

# Ejecutar la función del chatbot
consultar_chatbot()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Resultados más similares encontrados en el dataset:
Frase: Los sistemas de recomendación se basan en el aprendizaje automático. (Similitud: 15.865922927856445)
Frase: El Internet de las Cosas está cambiando la forma en que vivimos. (Similitud: 30.20244598388672)
Frase: La robótica es una combinación de ingeniería y tecnología avanzada. (Similitud: 33.447967529296875)
Consulta a BLOOMZ
Contexto: Los sistemas de recomendación se basan en el aprendizaje automático.. Ahora responde a la pregunta: ¿En qué se basan los sistemas de recomendación?


c:\Users\Marcos\Programas\Python3.8\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\Marcos\Programas\Python3.8\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\Marcos\Programas\Python3.8\lib\site-packages\transformers\generation\configuration_utils.py:649: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(



Respuesta generada por BLOOMZ:
Contexto: Los sistemas de recomendación se basan en el aprendizaje automático.. Ahora responde a la pregunta: ¿En qué se basan los sistemas de recomendación? aprendizaje
